# 1. Setup Instructions

Before executin the code, follow these steps for a smooth experience:

## I. Initial Setup and Preparation
- ### Selecting the Hardware Accelerator
  - Navigate to "Runtime" -> "Change runtime type" in the Colab menu.
  - Choose "GPU" as your hardware accelerator for optimal performance.

- ### Python Version Check
  - Execute `!python --version` in a new cell to confirm your Python version.
  - This notebook is optimized for Python 3.10.

- ### Accessing Necessary Files
  - All required files, including datasets and embeddings, are in our GitHub repository.
  - Download these files and upload them to your Google Drive in a structured directory, like "/content/drive/MyDrive/project/PalliativEtiological
/".

- ### Google Drive / Colab's local disk
  - Mount your Google Drive to access the files.
  - Alternatively, upload files to Colab's local disk.

- ### Run Code
  To run specific sections or the entire code efficiently, follow these guidelines:
  
  - **Initial Setup:**
    - Begin with the setup instructions to establish the necessary environment.
    - Make sure the paths for all required files are correctly set.

  - **Running Specific Sections (Sections 2-9):**
    - **File Dependencies:**
      - Check each section's instructions for its file requirements.
      - Ensure these files are loaded and their paths are correctly set.
    - **Independence of Sections:**
      - Sections can run independently, **except for section 7**, which needs the 10-fold CV SciBert. Confirm that all necessary files for each section are available.
    - **Note:**
      - Before running a specific section, it is essential to verify that all classes, methods, and files required by that section are correctly loaded and executed.


  - **Running the Entire Notebook:**
    - Go to "Runtime" in the Colab menu and select "Run all".
    - **Key File Check:** Ensure `comprehensive_dataset.csv.gz` is available and loaded, as it's critical for the entire notebook.

  - **Source of Files:**
    - Download necessary files from our GitHub repository.
    - Store them correctly in your Google Drive or Colab's local disk.



## **II. Using Pre-generated Resources**
- ### Pre-generated Embeddings
  - Generating embeddings for the GNN model can be quite time-consuming.
  - To expedite the process, you may choose to use pre-generated embeddings from our GitHub repository.
  - Instructions for using these embeddings are provided in the relevant sections of this notebook.

## **III. Fine-Tuned SciBERT Model**
  - Please note: To initialize the tokenizer for the SciBERT model, the use of an access token is optional. While an access token can be used (obtainable from your Hugging Face account), it's not mandatory for accessing public models like SciBERT. If you have an access token, assign it to the variable access_token. Otherwise, set the variable to None.

## **IV. Comparing GNN Model and SciBERT**
- ### Fine-Tuning for Comparison
  - To effectively compare the performance between the GNN model and SciBERT, it's essential to fine-tune both using the same 10-fold cross-validation process.
  - Detailed instructions for fine-tuning SciBERT using 10-fold cross-validation are provided in the respective sections of this notebook.

## **V. Note on Naming Conventions in Code**:
- In the development of our machine learning model, which we refer to as **DruGNNosis-MoA** in the paper, <br>
the code utilizes a more concise variable name and associated class name, **hetroSciGNN**. This decision was made to streamline the coding process and enhance readability during development.

In [ ]:
# check python version
!python --version

## Mounting Google Drive and Specifying Paths for Dataset, Model, and Embeddings

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # Mounting Google Drive for File Access

Specify filenames for the embeddings generated using SciBERT
Define the file paths for embeddings to be used as input in the GNN model.
Adjust the paths as per your directory structure.

In [ ]:
comprehensive_dataset_path = '/content/drive/MyDrive/project/GNN/comprehensive_dataset.csv.gz'  # Replace with your dataset path before running

cv_index_file_path = 'fold_indexes.pkl'

"""
Fine tuned SciBERT model
"""
# Define the path of the directory to be compressed
model_dir_format = 'fine_tuned_scibert_fold_{}' # Base directory name
compressed_model_format = 'fine_tuned_scibert_fold_{}.tar.gz' # Format for compressed model name

"""
Initialize the tokenizer for SciBERT. The 'token' argument is optional.
If you have an access token from Hugging Face (which can be generated from your Hugging Face account),
you can use it here by assigning it to 'access_token'.
However, for accessing public models like the one we use (SciBERT), it's optional.
"""
access_token = None # Replace with your Hugging Face access token, or set to None if you don't have one.

"""
Embeddings
"""
text_embeddings_file_path_format = 'text_embeddings_fold_{}.pkl'
eigen_featues_path = 'eigen_features_baseModel_combined.pt'
# Define the original and reduced embedding dimensions
original_embedding_dim = 768 # Do not change
reduced_embedding_dim = 128 # Do not change

"""
Paths for results metrices
"""
SciBERT_valid_fold_metrics_path = 'SciBERT_valid_fold_metrics.pkl'
hetroSciGNN_cv_results_path = 'DruGNNosis-MoA_cv_results.pkl'
baseGNN_cv_results_path = 'baseGNN_cv_results.pkl'
hetroSciGNN_test_metrics_path = 'DruGNNosis-MoA_test_metrics.pkl'

"""
Best model
"""
fine_tuned_90DataSet_path = 'fine_tuned_scibert_noValidSet'
compressed_bestModel_format = 'fine_tuned_scibert_noValidSet.tar.gz'

# The path for the text embeddings for the best model
best_model_gene_features_path =  'gene_embeddings_tensor_final.pt'
best_model_disease_features_path = 'disease_embeddings_tensor_final.pt'
best_model_drug_features_path = 'drug_embeddings_tensor_final.pt'

best_model_path = 'DruGNNosis-MoA_model.pth'
best_inputData_path = 'DruGNNosis-MoA_data.pkl'


## Installing Necessary Libraries


In [ ]:
!pip install torch_geometric

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers[torch]

## Importing Necessary Libraries

In [ ]:
# Standard libraries
import os
import csv
import tarfile
import gc
import random
import pickle
import gzip
import shutil
from itertools import product
from collections import defaultdict, Counter
from tqdm.notebook import tqdm

# Scientific computing and data manipulation
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs

# Machine Learning and Deep Learning frameworks
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

# Optimization algorithms from PyTorch
from torch.optim import Adam, SGD, AdamW, LBFGS, Adadelta, RMSprop
from torch.nn.utils import clip_grad_norm_
# PyTorch Geometric (PyG) for Graph Neural Networks (GNNs)
import torch_geometric
from torch_geometric.data import HeteroData
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import (GraphConv, GATConv, GCNConv, SAGEConv, GATv2Conv, Linear, HeteroConv, HGTConv, RGCNConv, RGATConv, MessagePassing, global_add_pool, Node2Vec)
import torch_geometric.transforms as T
from torch_geometric.explain import GNNExplainer
# scikit-learn
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, f1_score, classification_report, roc_auc_score, auc, accuracy_score, balanced_accuracy_score, precision_score, recall_score, roc_curve
from sklearn.utils.class_weight import compute_class_weight
from sklearn.decomposition import PCA

# Plotting and visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import networkx as nx
from IPython.display import Javascript  # Restrict height of output cell.

# NLP and transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, AutoModel
from datasets import Dataset, DatasetDict

Generate requirements.txt file

In [ ]:
import pkg_resources
import platform

# List of third-party libraries
libraries = [
    'numpy', 'pandas', 'scipy', 'torch', 'torch-geometric',
    'scikit-learn', 'seaborn', 'matplotlib', 'networkx',
    'transformers', 'datasets'
]

# Get the current Python version
python_version = platform.python_version()

# File to write the requirements
with open('requirements.txt', 'w') as f:

    f.write(f"# Python version: {python_version}\n")

    for lib in libraries:
        try:
            version = pkg_resources.get_distribution(lib).version
            f.write(f"{lib}=={version}\n")
        except pkg_resources.DistributionNotFound:
            f.write(f"# {lib} not found\n")

 ## Loading the Dataset

In [ ]:
# Load data sets that contain all the information we need for the graph.
df_all = pd.read_csv(comprehensive_dataset_path) # Check that the path for the dataset is correct

# Filter df that contain only drugs with e/p.
df = df_all[df_all['MoA'] != 'both']

# Reset the index of the DataFrame to ensure continuity after filtering rows.
df.reset_index(drop=True, inplace=True)

# 2. Preprocess The Dataset & Construct a Heterogeneous Network Using PyG

Setting Seed for Reproducibility

In [ ]:
def set_seed(seed_value=24):
    """Set seed for reproducibility."""
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    np.random.seed(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(24)  # Initialize the seed with a specific value (24) for reproducibility.


Constructing a Heterogeneous Network

In [ ]:
# 1. Create node mappings

# Drugs
unique_drugs = df.loc[df['Source Type'] == 'Drug', 'Source Name'].unique().tolist()
drug_mapping = {drug: i for i, drug in enumerate(unique_drugs)}

# Diseases
unique_diseases = df[df['Source Type'] == 'Disease']['Source Name'].unique().tolist()
disease_mapping = {disease: i for i, disease in enumerate(unique_diseases)}

# Genes (Include all genes)
genes_from_source = df[df['Source Type'] == 'Gene']['Source Name'].unique().tolist()
genes_from_target = df[df['Target Type'] == 'Gene']['Target Name'].unique().tolist()
all_genes = list(set(genes_from_source + genes_from_target))
gene_mapping = {gene: i for i, gene in enumerate(all_genes)}

# 2. Process node labels for drugs

# Extract unique drug information
drug_info_df = df[df['Source Type'] == 'Drug'].drop_duplicates(subset=['Source Name'])

# Convert MoA to labels
drug_info_df['MoA'] = drug_info_df['MoA'].apply(lambda x: 0 if x == 'p' else (1 if x == 'e' else x))

# Convert the 'MoA' column to a numpy array of type float32, then to a PyTorch tensor
drug_y = torch.tensor(drug_info_df['MoA'].values.astype(np.float32))

# Create a mask to identify which drugs have labels (MoA not equal to -1, can be usefull in the feature.)
drug_mask = drug_y != -1

# 3. Create edge indices for the various interactions

# Drug-Gene interactions
drug_gene_edges_df = df[(df['Source Type'] == 'Drug') & (df['Target Type'] == 'Gene')]
source_nodes_dg = drug_gene_edges_df["Source Name"].map(drug_mapping).dropna().astype(int).values
target_nodes_dg = drug_gene_edges_df['Target Name'].map(gene_mapping).dropna().astype(int).values
edge_index_dg = torch.tensor(np.array([source_nodes_dg, target_nodes_dg]), dtype=torch.long)

# Disease-Gene interactions
disease_gene_edges_df = df[(df['Source Type'] == 'Disease') & (df['Target Type'] == 'Gene')]
source_nodes_pg = disease_gene_edges_df['Source Name'].map(disease_mapping).dropna().astype(int).values
target_nodes_pg = disease_gene_edges_df['Target Name'].map(gene_mapping).dropna().astype(int).values
edge_index_pg = torch.tensor(np.array([source_nodes_pg, target_nodes_pg]), dtype=torch.long)

# Gene-Gene interactions (PPI)
gene_gene_edges_df = df[(df['Source Type'] == 'Gene') & (df['Target Type'] == 'Gene')]
source_nodes_gg = gene_gene_edges_df['Source Name'].map(gene_mapping).dropna().astype(int).values
target_nodes_gg = gene_gene_edges_df['Target Name'].map(gene_mapping).dropna().astype(int).values
edge_index_gg = torch.tensor(np.array([source_nodes_gg, target_nodes_gg]), dtype=torch.long)


Setting Up Train-Validation and Test Splits

In [ ]:
# Calculate total number of nodes
num_nodes_drugs = len(unique_drugs)

# Split the drugs into training/validation and test sets
train_val_drugs, test_drugs = train_test_split(unique_drugs, test_size=0.1, random_state=24, stratify=drug_info_df['MoA'])

# Get the indices for training/validation and test drugs
train_val_indices = drug_info_df[drug_info_df['Source Name'].isin(train_val_drugs)].index
test_indices = drug_info_df[drug_info_df['Source Name'].isin(test_drugs)].index

train_val_labels = drug_info_df.loc[train_val_indices, 'MoA'].values
test_labels = drug_info_df.loc[test_indices, 'MoA'].values

Setting up a Stratified K-Fold cross-validation process & saving the fold indices to a file.

In [ ]:
# Initialize Stratified K-Fold for the cross-validation and store fold indexes
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=24)
fold_indexes = [(train_idx, val_idx) for train_idx, val_idx in skf.split(train_val_indices, train_val_labels)]

# Save fold_indexes to a file
with open(cv_index_file_path, 'wb') as f:
    pickle.dump(fold_indexes, f)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Determine whether a GPU is available for computation.


# 3. SciBERT Model Classes & Methods
For more information on SciBERT, visit: https://github.com/allenai/scibert

### Defining Training Data, Tokenizer, Methods for Language Model

In [ ]:
def extract_embeddings(model, tokenizer, sentences):
    model.to(device)
    tokens = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)
    tokens = tokens.to(device)
    with torch.no_grad():
        outputs = model(**tokens)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu()

# Function to apply PCA and reduce dimensionality
def apply_pca(embeddings, n_components):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(embeddings)

# Function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions),
        'precision': precision_score(labels, predictions),
        'recall': recall_score(labels, predictions),
        'roc_auc': roc_auc_score(labels, logits[:,1])

    }

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")


In [ ]:
# Initialize lists to store probabilities, true labels, and metrics
all_true_labels = []
all_pred_probs = []
SciBERT_valid_fold_metrics = []

# Extract the "Drug_Description"
drug_info_df['text'] = drug_info_df['Drug_Description'].fillna('')

train_val_df = drug_info_df.iloc[train_val_indices].copy() # Create the train_valid df with the same 'test_indices' we used for the GNN.

train_val_df.rename(columns={'MoA': 'label'}, inplace=True)

# Tokenization - Using an access token is optional for public models - include it with token=access_token if you have one, or omit it otherwise.
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', token=access_token)

### SciBERT Training and Embedding Generation

In [ ]:
# Load fold_indexes from the file
with open(cv_index_file_path, 'rb') as f:
    fold_indexes = pickle.load(f)

for fold, (train_idx, val_idx) in enumerate(fold_indexes):

    print(f"Training for fold {fold}")

    print(torch.cuda.is_available())
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Determine whether a GPU is available for computation.

    # Splitting data into train and validation for current fold
    train_df = train_val_df.iloc[train_idx]
    val_df = train_val_df.iloc[val_idx]

    # Create datasets
    train_dataset = Dataset.from_pandas(train_df).map(preprocess_function, batched=True)
    val_dataset = Dataset.from_pandas(val_df).map(preprocess_function, batched=True)

    # Model and Training
    model_LLM = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=2, token=access_token)

    training_args = TrainingArguments(
        output_dir=f'.llm_model_output/llm_model_output_fold_{fold}',
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.002,
        evaluation_strategy="epoch"
    )

    trainer = Trainer(
        model=model_LLM,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate on validation set
    valid_results = trainer.evaluate(val_dataset)
    SciBERT_valid_fold_metrics.append(valid_results)

    # Get predictions for ROC curve analysis
    predictions = trainer.predict(val_dataset)
    logits = predictions.predictions
    true_labels = predictions.label_ids
    softmax = torch.nn.Softmax(dim=1)
    pred_probs = softmax(torch.from_numpy(logits)).numpy()[:, 1]  # Assuming class 1 is the positive class
    all_true_labels.extend(true_labels)
    all_pred_probs.extend(pred_probs)


    dir_fine_tuned_scibert_current_fold = model_dir_format.format(fold)
    compressed_model_name = compressed_model_format.format(fold)


    model_LLM.save_pretrained(dir_fine_tuned_scibert_current_fold)

    # Compress the model directory
    with tarfile.open(compressed_model_name, "w:gz") as tar:
        tar.add(dir_fine_tuned_scibert_current_fold, arcname=os.path.basename(dir_fine_tuned_scibert_current_fold))

    # Delete the uncompressed model directory
    shutil.rmtree(dir_fine_tuned_scibert_current_fold)

    # Clear the CUDA cache if using GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Explicitly invoke garbage collection
    gc.collect()

# Save metrics and predictions for later analysis
with open(SciBERT_valid_fold_metrics_path, 'wb') as file:
    pickle.dump({
        'metrics': SciBERT_valid_fold_metrics,
        'true_labels': all_true_labels,
        'pred_probs': all_pred_probs
    }, file)

# 4. Graph Neural Network (Baseline & DruGNNosis-MoA models)

## Methods: Generate new HetroData object, Training & Evalution for GNNs.

In [ ]:
transform = T.Compose([T.ToUndirected()])

# Function to rebuild the graph
def rebuild_graph(df, drug_mapping, gene_mapping, disease_mapping, drug_features, gene_features, disease_features, drug_y):
    data = HeteroData() # Create the HeteroData object will be used to store the heterogeneous graph

    data['gene'].x = gene_features # Set gene features
    data['disease'].x = disease_features # Set disease features
    data['drug'].x = drug_features # Set drug features

    data['gene'].num_nodes = len(gene_mapping)
    data['drug'].num_nodes = len(drug_mapping)
    data['disease'].num_nodes = len(disease_mapping)

    data['drug'].y = drug_y # Set drug labels.
    data['drug', 'drug_target', 'gene'].edge_index = edge_index_dg # Set drug-gene edges
    data['disease', 'disease_gene', 'gene'].edge_index = edge_index_pg # Set phenotype-gene edges
    data['gene', 'PPI', 'gene'].edge_index = edge_index_gg # Set gene-gene edges

    data = transform(data) # Set reverse edges.
    data.validate(raise_on_error=True)  # Check the graph structure

    return data

Print the topological features of the heterogeneous graph (without including features).

In [ ]:
print(rebuild_graph(df, drug_mapping, gene_mapping, disease_mapping, _, _, _,  drug_y))

In [ ]:
def train(model, optimizer, data, x_dict, edge_index_dict):
    # sets the model to training mode.
    model.train().to(device)
    # Before a new forward and backward pass, we need to ensures that gradients from the previous iterations don't accumulate.
    optimizer.zero_grad()
    # Performs a forward pass through the model.
    out = model(x_dict, edge_index_dict)

    # Flatten the output and labels for BCELoss
    out = out[data['drug'].train_mask].squeeze()
    labels = data['drug'].y[data['drug'].train_mask].type_as(out)

    criterion = torch.nn.BCELoss()
    loss = criterion(out, labels)

    loss.backward() # computes the gradients of the loss with respect to the model parameters. It's the step where backpropagation happens.
    clip_grad_norm_(model.parameters(), max_norm=0.5)
    optimizer.step() # updates the model's parameters based on the computed gradients.

    # Balanced Accuracy Calculation
    y_true_train = labels.cpu().detach().numpy()
    y_pred_train = (out.cpu().detach().numpy() > 0.5).astype(int) # Threshold to get 0 or 1 as class labels

    balanced_acc_train = balanced_accuracy_score(y_true_train, y_pred_train)

    correct = y_pred_train == y_true_train
    acc_train = correct.sum() / len(y_true_train)

    return float(loss), acc_train, balanced_acc_train

@torch.no_grad()
def evaluate(model, data, x_dict, edge_index_dict, mask_type='valid'):
    model.eval().to(device)
    out = model(x_dict, edge_index_dict)

    # Choose the correct mask based on the function call
    if mask_type == 'valid':
        mask = data['drug'].val_mask
    elif mask_type == 'test':
        mask = data['drug'].test_mask
    else:
        raise ValueError("Invalid mask type specified")

    labels = data['drug'].y[mask].type_as(out)
    y_true = labels.cpu().detach().numpy()
    y_true = y_true.astype('int32')

    masked_out = out[mask]
    y_pred_prob = masked_out.cpu().detach().numpy().flatten()

    y_pred = (y_pred_prob > 0.5).astype(int)
    y_pred = y_pred.flatten()

    # Compute metrics
    f_score = f1_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    precision = precision_score(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_pred_prob)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    correct = y_pred == y_true
    acc = correct.sum() / len(y_true)
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)

    return acc, balanced_acc, recall, precision, f_score, roc_auc, fpr, tpr

## DruGNNosis-MoA Model

DruGNNosis-MoA object model

In [ ]:
class HeteroSciGNN(torch.nn.Module):
    def __init__(self):
        super(HeteroSciGNN, self).__init__()
        # in_channels parameter specifies the dimensionality of the input features of the source nodes.
        input_dim = 128
        layer1_output_dim = 512
        model_output_dim = 1

        # First layer with SAGEConv.
        self.layer1 = HeteroConv({
            ('drug', 'drug_target', 'gene'): SAGEConv(in_channels=input_dim, out_channels=layer1_output_dim),
            ('disease', 'disease_gene', 'gene'): SAGEConv(in_channels=input_dim, out_channels=layer1_output_dim),
            ('gene', 'PPI', 'gene'): SAGEConv(in_channels=input_dim, out_channels=layer1_output_dim),
            ('gene', 'rev_drug_target', 'drug'): SAGEConv(in_channels=input_dim, out_channels=layer1_output_dim),
            ('gene', 'rev_disease_gene', 'disease'): SAGEConv(in_channels=input_dim, out_channels=layer1_output_dim)
        })

        # Second layer with SAGEConv.
        self.layer2 = HeteroConv({
            ('drug', 'drug_target', 'gene'): SAGEConv(in_channels=layer1_output_dim, out_channels=model_output_dim),
            ('disease', 'disease_gene', 'gene'): SAGEConv(in_channels=layer1_output_dim, out_channels=model_output_dim),
            ('gene', 'PPI', 'gene'): SAGEConv(in_channels=layer1_output_dim, out_channels=model_output_dim),
            ('gene', 'rev_drug_target', 'drug'): SAGEConv(in_channels=layer1_output_dim, out_channels=model_output_dim),
            ('gene', 'rev_disease_gene', 'disease'): SAGEConv(in_channels=layer1_output_dim, out_channels=model_output_dim)
        })

        self.sigmoid = torch.nn.Sigmoid() # Sigmoid activation for binary classification [0, 1].
        self.relu = torch.nn.ReLU() # Non linear activation function.

        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, x_dict, edge_index_dict, return_intermediate=False):
        """
        x_dict -  A dictionary where keys are node types (e.g., 'drug', 'gene', 'disease') and values are node feature tensors.
        edge_index_dict - A dictionary where keys are edge relations (e.g., ('drug', 'drug_target', 'gene'))
                            and values are edge index tensors indicating source and destination nodes for each edge.
        """

        # First layer with dropout
        x_dict = self.layer1(x_dict, edge_index_dict)
        for key in x_dict:
            x_dict[key] = self.dropout(self.relu(x_dict[key]))


        if return_intermediate: # Used for T-SNE
            # x_dict = self.layerTSNE(x_dict, edge_index_dict)
            return x_dict['drug']


        # Second layer with dropout
        x_dict = self.layer2(x_dict, edge_index_dict)


        # Directly use the output from layer2
        out = self.sigmoid(x_dict['drug'])

        return out

Text embedding using the finetuned SciBert per fold.

In [ ]:
"""
# --------------- Extract Embeddings with Current Fine-Tuned Model ---------------
"""
# Load fold_indexes from the file
with open(cv_index_file_path, 'rb') as f:
    fold_indexes = pickle.load(f)

for fold, _ in enumerate(fold_indexes):
    print(f"Embeddings for fold {fold}: ")

    # Generate the directory and compressed model for current fold
    dir_fine_tuned_scibert_current_fold = model_dir_format.format(fold)
    compressed_model_name = compressed_model_format.format(fold)

    # Decompress the model
    with tarfile.open(compressed_model_name, 'r:gz') as tar:
        tar.extractall()

    # Load the fine-tuned model weights into a base AutoModel
    model_LLM = AutoModel.from_pretrained(dir_fine_tuned_scibert_current_fold)

    # Initialize lists to store embeddings
    drug_embeddings = []

    for drug_description in tqdm(drug_info_df['Drug_Description'], desc="Generate Drug Embedding: "):
        embedding = extract_embeddings(model_LLM, tokenizer, drug_description)
        drug_embeddings.append(embedding)

    description_embeddings_tensor = torch.cat(drug_embeddings, dim=0)

    gene_embeddings = []

    # Generate gene embeddings
    for gene in tqdm(all_genes, desc="Generate Gene Embedding: "):
        embedding = extract_embeddings(model_LLM, tokenizer, gene)
        gene_embeddings.append(embedding)

    # Convert list of tensors to a single tensor
    gene_embeddings_tensor = torch.cat(gene_embeddings, dim=0)

    disease_embeddings = []

    # Generate disease embeddings
    for disease in tqdm(unique_diseases, desc="Generate Disease Embedding: "):
        embedding = extract_embeddings(model_LLM, tokenizer, disease)
        disease_embeddings.append(embedding)

    # Convert list of tensors to a single tensor
    disease_embeddings_tensor = torch.cat(disease_embeddings, dim=0)

    # Applying PCA to reduce the dimensionality of embeddings
    gene_embeddings_reduced = apply_pca(gene_embeddings_tensor, reduced_embedding_dim)
    disease_embeddings_reduced = apply_pca(disease_embeddings_tensor, reduced_embedding_dim)
    drug_embeddings_reduced = apply_pca(description_embeddings_tensor, reduced_embedding_dim)

    # Updating the graph with reduced embeddings
    gene_features_pca = torch.tensor(gene_embeddings_reduced, dtype=torch.float32)
    disease_features_pca = torch.tensor(disease_embeddings_reduced, dtype=torch.float32)
    drug_features_pca = torch.tensor(drug_embeddings_reduced, dtype=torch.float32)

    # Combine embeddings into a dictionary
    embeddings_dict = {
        'gene_embeddings': gene_features_pca,
        'disease_embeddings': disease_features_pca,
        'drug_embeddings': drug_features_pca
    }

    text_embeddings_file_path = text_embeddings_file_path_format.format(fold)

    # Save the dictionary to a file
    with open(text_embeddings_file_path, 'wb') as f:
        pickle.dump(embeddings_dict, f)

Training Loop

In [ ]:
# Load fold_indexes from the file
with open(cv_index_file_path, 'rb') as f:
    fold_indexes = pickle.load(f)

# Initialize a DataFrame to hold all results
hetroSciGNN_cv_df = pd.DataFrame()

for fold, (train_idx, val_idx) in enumerate(fold_indexes):

    text_embeddings_file_path = text_embeddings_file_path_format.format(fold)

    # Load the embeddings dictionary
    with open(text_embeddings_file_path, 'rb') as f:
        embeddings_dict = pickle.load(f)

    # Access the embeddings
    gene_features_pca = embeddings_dict['gene_embeddings']
    disease_features_pca = embeddings_dict['disease_embeddings']
    drug_features_pca = embeddings_dict['drug_embeddings']

    # Rebuild the graph for the current fold
    hetroSciGNN_data = rebuild_graph(df, drug_mapping, gene_mapping, disease_mapping, drug_features_pca, gene_features_pca, disease_features_pca,  drug_y)
    hetroSciGNN_model = HeteroSciGNN()

    # The Adam optimizer is initialized with a learning rate.
    hetroSciGNN_optimizer = Adam(hetroSciGNN_model.parameters(), lr=0.0006)

    hetroSciGNN_data = hetroSciGNN_data.to(device)
    hetroSciGNN_model = hetroSciGNN_model.to(device) # Move the hetroSciGNN_model to the available device (either GPU or CPU).

    # Create masks for the current fold
    train_mask = torch.zeros(num_nodes_drugs, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes_drugs, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes_drugs, dtype=torch.bool)

    test_mask[test_indices] = True
    train_mask[train_idx] = True
    val_mask[val_idx] = True

    # Update the data object with train, validation, and test masks for drug nodes
    hetroSciGNN_data['drug'].train_mask = train_mask
    hetroSciGNN_data['drug'].val_mask = val_mask
    hetroSciGNN_data['drug'].test_mask = test_mask

    # Train and validate the hetroSciGNN_model using the current fold
    for epoch in range(1, 36):
        loss, acc_train, balanced_acc_train = train(hetroSciGNN_model, hetroSciGNN_optimizer, hetroSciGNN_data, hetroSciGNN_data.x_dict, hetroSciGNN_data.edge_index_dict)
        acc_v, balanced_acc_v, recall_v, precision_v, f1_v, roc_auc_v, fpr_v, tpr_v = evaluate(hetroSciGNN_model, hetroSciGNN_data, hetroSciGNN_data.x_dict, hetroSciGNN_data.edge_index_dict, mask_type='valid')
        print(f'Fold: {fold}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc_train: {acc_train:.4f}, Acc_valid: {acc_v:.4f}, F1-Score_valid: {f1_v:.4f}, Roc-Auc_valid: {roc_auc_v:.4f}')
        if (f1_v >= 0.95):
            break

    # Save metrics for the validation set of the current fold.
    hetroSciGNN_fold_metrics_valid = {
        'fold': fold,
        'acc_v': acc_v,
        'balanced_acc_v': balanced_acc_v,
        'recall_v': recall_v,
        'precision_v': precision_v,
        'f1_v': f1_v,
        'roc_auc_v': roc_auc_v,
        'fpr': fpr_v,
        'tpr': tpr_v
    }

    hetroSciGNN_fold_results = pd.DataFrame([hetroSciGNN_fold_metrics_valid])
    hetroSciGNN_cv_df = pd.concat([hetroSciGNN_cv_df, hetroSciGNN_fold_results], ignore_index=True)

    # After using the model and data
    del hetroSciGNN_model  # Delete the model
    del hetroSciGNN_data   # Delete the data

    # Clear the CUDA cache if using GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Explicitly invoke garbage collection.
    gc.collect()

# Save the results DataFrame to a file
hetroSciGNN_cv_df.to_pickle(hetroSciGNN_cv_results_path)

## Baseline GNN model

baseGNN object model

In [ ]:
class baseGNN(torch.nn.Module):
    def __init__(self):
        super(baseGNN, self).__init__()

        input_dim = 16
        layer1_output_dim = 256
        model_output_dim = 1

        # First layer with RGCNConv.
        self.layer1 = HeteroConv({
            ('drug', 'drug_target', 'gene'): SAGEConv(in_channels=input_dim, out_channels=layer1_output_dim),
            ('disease', 'disease_gene', 'gene'): SAGEConv(in_channels=input_dim, out_channels=layer1_output_dim),
            ('gene', 'PPI', 'gene'): SAGEConv(in_channels=input_dim, out_channels=layer1_output_dim),
            ('gene', 'rev_drug_target', 'drug'): SAGEConv(in_channels=input_dim, out_channels=layer1_output_dim),
            ('gene', 'rev_disease_gene', 'disease'): SAGEConv(in_channels=input_dim, out_channels=layer1_output_dim)
        })
        self.layer2 = HeteroConv({
            ('drug', 'drug_target', 'gene'): SAGEConv(in_channels=layer1_output_dim, out_channels=model_output_dim),
            ('disease', 'disease_gene', 'gene'): SAGEConv(in_channels=layer1_output_dim, out_channels=model_output_dim),
            ('gene', 'PPI', 'gene'): SAGEConv(in_channels=layer1_output_dim, out_channels=model_output_dim),
            ('gene', 'rev_drug_target', 'drug'): SAGEConv(in_channels=layer1_output_dim, out_channels=model_output_dim),
            ('gene', 'rev_disease_gene', 'disease'): SAGEConv(in_channels=layer1_output_dim, out_channels=model_output_dim)
        })

        self.sigmoid = torch.nn.Sigmoid() # Sigmoid activation for binary classification [0, 1].
        self.relu = torch.nn.ReLU() # Non linear activation function.

    def forward(self, x_dict, edge_index_dict, return_intermediate=False):
        """
        x_dict -  A dictionary where keys are node types (e.g., 'drug', 'gene', 'disease') and values are node feature tensors.
        edge_index_dict - A dictionary where keys are edge relations (e.g., ('drug', 'drug_target', 'gene'))
                            and values are edge index tensors indicating source and destination nodes for each edge.
        """

        # First layer
        x_dict = self.layer1(x_dict, edge_index_dict)
        for key in x_dict:
            x_dict[key] = self.relu(x_dict[key])

        # Second layer
        x_dict = self.layer2(x_dict, edge_index_dict)

        # Directly use the output from layer4
        out = self.sigmoid(x_dict['drug'])

        return out

 Embedding Eigen Vector for the baseline gnn

In [ ]:
# Calculating the total number of nodes in the graph
total_nodes = len(drug_mapping) + len(disease_mapping) + len(gene_mapping)
combined_adj_matrix = torch.zeros((total_nodes, total_nodes))

# Adding Drug-Gene edges
for src, tgt in tqdm(zip(source_nodes_dg, target_nodes_dg), total=len(source_nodes_dg), desc="Adding Drug-Gene Edges"):
    # Since drug nodes come first, their indices are unchanged
    # Gene indices need to be offset by the number of drug and phenotype nodes
    combined_adj_matrix[src, len(drug_mapping) + len(disease_mapping) + tgt] = 1
    combined_adj_matrix[len(drug_mapping) + len(disease_mapping) + tgt, src] = 1

# Adding Phenotype-Gene edges
for src, tgt in tqdm(zip(source_nodes_pg, target_nodes_pg), total=len(source_nodes_pg), desc="Adding Phenotype-Gene Edges"):
    # Offset source index for phenotypes by the number of drugs
    # Offset target index for genes as before
    combined_adj_matrix[len(drug_mapping) + src, len(drug_mapping) + len(disease_mapping) + tgt] = 1
    combined_adj_matrix[len(drug_mapping) + len(disease_mapping) + tgt, len(drug_mapping) + src] = 1

# Adding Gene-Gene edges (PPI)
for src, tgt in tqdm(zip(source_nodes_gg, target_nodes_gg), total=len(source_nodes_gg), desc="Adding Gene-Gene Edges"):
    # Both source and target indices need to be offset for genes
    combined_adj_matrix[len(drug_mapping) + len(disease_mapping) + src, len(drug_mapping) + len(disease_mapping) + tgt] = 1
    combined_adj_matrix[len(drug_mapping) + len(disease_mapping) + tgt, len(drug_mapping) + len(disease_mapping) + src] = 1

def compute_eigen_features(adj_matrix, k=16):
    # Convert to CSR format for efficient computation
    adj_matrix_csr = csr_matrix(adj_matrix.numpy())

    # Compute the top k eigenvalues and eigenvectors
    _, eigenvectors = eigs(adj_matrix_csr, k=k, which='LM')
    eigenvectors = eigenvectors.real  # Take the real values

    # Z-score normalization
    eigenvector_means = np.mean(eigenvectors, axis=0)
    eigenvector_stds = np.std(eigenvectors, axis=0)
    normalized_eigenvectors = (eigenvectors - eigenvector_means) / eigenvector_stds

    return torch.from_numpy(normalized_eigenvectors)

# Compute eigen features for the combined graph
combined_eigen_features = compute_eigen_features(combined_adj_matrix)

torch.save(combined_eigen_features, eigen_featues_path)

Training loop

In [ ]:
# Load fold_indexes from the file
with open(cv_index_file_path, 'rb') as f:
    fold_indexes = pickle.load(f)

# Initialize a DataFrame to hold all results
baseGNN_cv_df = pd.DataFrame()

for fold, (train_idx, val_idx) in enumerate(fold_indexes):

    combined_eigen_features = torch.load(eigen_featues_path)
    # Assign eigen features to each node type
    drug_baseModel_features = combined_eigen_features[:len(drug_mapping), :]
    disease_baseModel_features = combined_eigen_features[len(drug_mapping):(len(drug_mapping) + len(disease_mapping)), :]
    gene_baseModel_features = combined_eigen_features[(len(drug_mapping) + len(disease_mapping)):, :]

    # Rebuild the graph for the current fold
    baseGNN_data = rebuild_graph(df, drug_mapping, gene_mapping, disease_mapping, drug_baseModel_features, gene_baseModel_features, disease_baseModel_features, drug_y)
    baseGNN_model = baseGNN()

    # The Adam optimizer is initialized with a learning rate.
    baseGNN_optimizer = Adam(baseGNN_model.parameters(), lr=0.0005)

    print(torch.cuda.is_available())
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Determine whether a GPU is available for computation.
    baseGNN_data = baseGNN_data.to(device)
    baseGNN_model = baseGNN_model.to(device) # Move the model to the available device (either GPU or CPU).

    # Create masks for the current fold
    train_mask = torch.zeros(num_nodes_drugs, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes_drugs, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes_drugs, dtype=torch.bool)

    test_mask[test_indices] = True
    train_mask[train_idx] = True
    val_mask[val_idx] = True

    # Update the data object with train, validation, and test masks for drug nodes
    baseGNN_data['drug'].train_mask = train_mask
    baseGNN_data['drug'].val_mask = val_mask
    baseGNN_data['drug'].test_mask = test_mask

    # Train and validate the model using the current fold
    for epoch in range(1, 101):
        loss, acc_train, balanced_acc_train = train(baseGNN_model, baseGNN_optimizer, baseGNN_data, baseGNN_data.x_dict, baseGNN_data.edge_index_dict)
        acc_v, balanced_acc_v, recall_v, precision_v, f1_v, roc_auc_v, fpr_v, tpr_v = evaluate(baseGNN_model, baseGNN_data, baseGNN_data.x_dict, baseGNN_data.edge_index_dict, mask_type='valid')
        print(f'Fold: {fold}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc_train: {acc_train:.4f}, Acc_valid: {acc_v:.4f}, F1-Score_valid: {f1_v:.4f}, Roc-Auc_valid: {roc_auc_v:.4f}')

    # Save metrics for the validation set of the current fold.
    baseGNN_fold_metrics_valid = {
        'fold': fold,
        'acc_v': acc_v,
        'balanced_acc_v': balanced_acc_v,
        'recall_v': recall_v,
        'precision_v': precision_v,
        'f1_v': f1_v,
        'roc_auc_v': roc_auc_v,
        'fpr': fpr_v,
        'tpr': tpr_v
    }

    # Inside your fold loop
    baseGNN_fold_results = pd.DataFrame([baseGNN_fold_metrics_valid])
    baseGNN_cv_df = pd.concat([baseGNN_cv_df, baseGNN_fold_results], ignore_index=True)


    del baseGNN_model  # Delete the model
    del baseGNN_data   # Delete the data

    # Clear the CUDA cache if using GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Explicitly invoke garbage collection
    gc.collect()

# Save the DataFrame to a file
baseGNN_cv_df.to_pickle(baseGNN_cv_results_path)

# 5. Compute the average on the validtion set.

SciBERT

In [ ]:
# Load the saved metrics
with open(SciBERT_valid_fold_metrics_path, 'rb') as file:
    SciBERT_valid_metrics_loaded = pickle.load(file)

# Extract the list of dictionaries containing the metrics
metrics_list = SciBERT_valid_metrics_loaded['metrics']

# Convert loaded metrics to a DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Print metrics for each fold for validation set in the desired format
print("Validation Metrics for each fold: \n")
for i, row in metrics_df.iterrows():
    print(f"Fold {i+1}: ", end='')
    print(', '.join([f"{metric}: {row[metric]:.4f}" for metric in metrics_df.columns]))

# Calculate average and standard deviation of metrics across folds for validation set
average_metrics = metrics_df.mean()
std_metrics = metrics_df.std()

# Print average and standard deviation of metrics in the desired format
print("\nAverage Validation Metrics Across All Folds:")
print(', '.join([f"\n Average {metric}: {average_metrics[metric]:.4f}" for metric in average_metrics.index]))

print("\nStandard Deviation of Validation Metrics Across All Folds:")
print(', '.join([f"\n Std {metric}: {std_metrics[metric]:.4f}" for metric in std_metrics.index]))

DruGNNosis-MoA Model

In [ ]:
# Load validation fold metrics
hetroSciGNN_cv_df = pd.read_pickle(hetroSciGNN_cv_results_path)

# After the loop, print metrics for each fold
print("Metrics for each fold:")
for index, row in hetroSciGNN_cv_df.iterrows():
    # Exclude the first 'fold' column and the last two columns from the row
    metrics_to_print = row[1:-2]
    print(f'Fold {index + 1}:', ', '.join([f'{key}: {value:.4f}' for key, value in metrics_to_print.items() if key != 'fold']))

# Calculate the average and standard deviation of metrics (only numeric columns)
average_metrics = hetroSciGNN_cv_df.mean(numeric_only=True)
std_metrics = hetroSciGNN_cv_df.std(numeric_only=True)

# Print the average results
print("\nAverage validation metrics across all folds:")
print(average_metrics[1:])

# Print the standard deviation of metrics
print("\nStandard deviation of validation metrics across all folds:")
print(std_metrics[1:])

BaseGNN model

In [ ]:
# Load validation fold metrics
baseGNN_cv_df = pd.read_pickle(baseGNN_cv_results_path)

# After the loop, print metrics for each fold
print("Metrics for each fold:")
for index, row in baseGNN_cv_df.iterrows():
    # Exclude the first 'fold' column and the last two columns from the row
    metrics_to_print = row[1:-2]
    print(f'Fold {index + 1}:', ', '.join([f'{key}: {value:.4f}' for key, value in metrics_to_print.items() if key != 'fold']))

# Calculate the average and standard deviation of metrics (only numeric columns)
average_metrics = baseGNN_cv_df.mean(numeric_only=True)
std_metrics = baseGNN_cv_df.std(numeric_only=True)

# Print the average results
print("\nAverage validation metrics across all folds:")
print(average_metrics[1:])

# Print the standard deviation of metrics
print("\nStandard deviation of validation metrics across all folds:")
print(std_metrics[1:])

# 6. Best model Training and Evaluation

## Train DruGNNosis-MoA model without validtion set

In [ ]:
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Determine whether a GPU is available for computation.

# Tokenization - Using an access token is optional for public models - include it with token=access_token if you have one, or omit it otherwise.
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', token=access_token)

# Initialize pre-trained model for scientific text.
model_LLM_embedding = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=2, token=access_token)

# Extract the "Drug_Description".
drug_info_df['text'] = drug_info_df['Drug_Description'].fillna('')

train_val_df = drug_info_df.iloc[train_val_indices].copy() # Create the train_valid df with the same 'test_indices' we used for the GNN.

train_val_df.rename(columns={'MoA': 'label'}, inplace=True)

# Create train data set without leaving data for validtion.
train90_dataset = Dataset.from_pandas(train_val_df).map(preprocess_function, batched=True)

In [ ]:
training_args = TrainingArguments(
  output_dir=f'./llm_model_output',
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=3,
  weight_decay=0.002,
  evaluation_strategy="no"
)

trainer = Trainer(
  model=model_LLM_embedding,
  args=training_args,
  train_dataset=train90_dataset,
  tokenizer=tokenizer,
  compute_metrics=None
)

# Train the model
trainer.train()

model_LLM_embedding.save_pretrained(fine_tuned_90DataSet_path)

# Create a tar.gz file
with tarfile.open(compressed_bestModel_format, "w:gz") as tar:
    tar.add(fine_tuned_90DataSet_path, arcname=os.path.basename(fine_tuned_90DataSet_path))

 Extract Embeddings with The Fine-Tuned Model

In [ ]:
print('\n Calculate Text Embedding For The SciBretGNN Model: \n')

# Load the fine-tuned model weights into a base AutoModel
model_LLM_embedding = AutoModel.from_pretrained(fine_tuned_90DataSet_path)

# Initialize lists to store embeddings
drug_embeddings = []

for drug_description in tqdm(drug_info_df['Drug_Description'], desc="Generate Drug Embedding: "):
    embedding = extract_embeddings(model_LLM_embedding, tokenizer, drug_description)
    drug_embeddings.append(embedding)

description_embeddings_tensor = torch.cat(drug_embeddings, dim=0)

gene_embeddings = []

# Generate gene embeddings
for gene in tqdm(all_genes, desc="Generate Gene Embedding: "):
    embedding = extract_embeddings(model_LLM_embedding, tokenizer, gene)
    gene_embeddings.append(embedding)

# Convert list of tensors to a single tensor
gene_embeddings_tensor = torch.cat(gene_embeddings, dim=0)

disease_embeddings = []

# Generate disease embeddings
for disease in tqdm(unique_diseases, desc="Generate Disease Embedding: "):
    embedding = extract_embeddings(model_LLM_embedding, tokenizer, disease)
    disease_embeddings.append(embedding)

# Convert list of tensors to a single tensor
disease_embeddings_tensor = torch.cat(disease_embeddings, dim=0)

# Applying PCA to reduce the dimensionality of embeddings
gene_embeddings_reduced = apply_pca(gene_embeddings_tensor, reduced_embedding_dim)
disease_embeddings_reduced = apply_pca(disease_embeddings_tensor, reduced_embedding_dim)
drug_embeddings_reduced = apply_pca(description_embeddings_tensor, reduced_embedding_dim)

# Updating the graph with reduced embeddings
gene_features_pca = torch.tensor(gene_embeddings_reduced, dtype=torch.float32)
disease_features_pca = torch.tensor(disease_embeddings_reduced, dtype=torch.float32)
drug_features_pca = torch.tensor(drug_embeddings_reduced, dtype=torch.float32)

# Save the embeddings
torch.save(gene_features_pca, best_model_gene_features_path)
torch.save(disease_features_pca, best_model_disease_features_path)
torch.save(drug_features_pca, best_model_drug_features_path)

DruGNNosis-MoA Model Training

In [ ]:
bestModel_gene_features = torch.load(best_model_gene_features_path)
bestModel_disease_features = torch.load(best_model_disease_features_path)
bestModel_drug_features = torch.load(best_model_drug_features_path)

# Rebuild the graph
hetroSciGNN_data = rebuild_graph(df, drug_mapping, gene_mapping, disease_mapping, bestModel_drug_features, bestModel_gene_features, bestModel_disease_features,  drug_y)
hetroSciGNN_model = HeteroSciGNN()

# The Adam optimizer is initialized with a learning rate.
hetroSciGNN_optimizer = Adam(hetroSciGNN_model.parameters(), lr=0.0006)

hetroSciGNN_data = hetroSciGNN_data.to(device)
hetroSciGNN_model = hetroSciGNN_model.to(device) # Move the hetroSciGNN_model to the available device (either GPU or CPU).

# Create masks
train_mask = torch.zeros(num_nodes_drugs, dtype=torch.bool)
test_mask = torch.zeros(num_nodes_drugs, dtype=torch.bool)

test_mask[test_indices] = True
train_mask[train_val_indices] = True

# Update the data object with train, and test masks for drug nodes
hetroSciGNN_data['drug'].train_mask = train_mask
hetroSciGNN_data['drug'].test_mask = test_mask

# Train the hetroSciGNN model
for epoch in range(1, 36):
    loss, acc_train, balanced_acc_train = train(hetroSciGNN_model, hetroSciGNN_optimizer, hetroSciGNN_data, hetroSciGNN_data.x_dict, hetroSciGNN_data.edge_index_dict)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc_train: {acc_train:.4f}')

# Save the model
torch.save(hetroSciGNN_model.state_dict(), best_model_path)

# Save the input data
with open(best_inputData_path, 'wb') as f:
    pickle.dump(hetroSciGNN_data, f)

## Evaluate the DruGNNosis-MoA on the test set

If you run this part separately, <br>
please ensure that you have run the 'HeteroSciGNN' class and the evaluate method beforehand and that your file paths are correct.

In [ ]:
# Load the model
hetroSciGNN_model = HeteroSciGNN()  # Initialize the model again
hetroSciGNN_model.load_state_dict(torch.load(best_model_path))
hetroSciGNN_model = hetroSciGNN_model.to(device)  # Move to the appropriate device

# Load the input data
with open(best_inputData_path, 'rb') as f:
    hetroSciGNN_data = pickle.load(f)

acc_t, balanced_acc_t, recall_t, precision_t, f1_t, auc_roc_t, fpr_t, tpr_t = evaluate(hetroSciGNN_model, hetroSciGNN_data, hetroSciGNN_data.x_dict, hetroSciGNN_data.edge_index_dict, mask_type='test')

# Save metrics for the test set current fold.
hetroSciGNN_metrics_test = {
    'acc_t': acc_t,
    'balanced_acc_t': balanced_acc_t,
    'recall_t': recall_t,
    'precision_t': precision_t,
    'f1_t': f1_t,
    'roc_auc_t': auc_roc_t,
    'fpr': fpr_t,
    'tpr': tpr_t
}

print(f'### TEST RESULTS ###')
print(f'Acc_test: {acc_t:.4f}, Recall: {recall_t:.4f}, Precision: {precision_t:.4f}, F1: {f1_t:.4f}, AUC: {auc_roc_t:.4f}')

# Save test results in a file
with open(hetroSciGNN_test_metrics_path, 'wb') as f:
    pickle.dump(hetroSciGNN_metrics_test, f)

# 7. Comparison of F1 and ROC-AUC Scores Across Models

In [ ]:
hetroSciGNN_cv_df = pd.read_pickle(hetroSciGNN_cv_results_path) # Load data for hetroSciGNN model
baseGNN_cv_df = pd.read_pickle(baseGNN_cv_results_path) # Load data for baseGNN model

# Load data for SciBERT model
with open(SciBERT_valid_fold_metrics_path, 'rb') as file:
    scibert_results = pickle.load(file)

# Extract F1 scores from hetroSciGNN
hetroSciGNN_f1_scores = hetroSciGNN_cv_df['f1_v']
print("DruGNNosis-MoA F1 Scores:")
print(hetroSciGNN_f1_scores)

# Extract F1 scores from baseGNN
baseGNN_f1_scores = baseGNN_cv_df['f1_v']
print("\nbaseGNN F1 Scores:")
print(baseGNN_f1_scores)

# Extract F1 scores for SciBERT
SciBERT_f1_scores = [fold_metrics['eval_f1'] for fold_metrics in scibert_results['metrics']]
print("SciBERT F1 Scores:")
print(SciBERT_f1_scores)

## 7.I. F1 Score Evaluation

In [ ]:
# Shapiro-Wilk Test for normality
gnn_f1_normality = stats.shapiro(hetroSciGNN_f1_scores)
scibert_f1_normality = stats.shapiro(SciBERT_f1_scores)

# Print normality test results
print("\nGNN F1 Normality Test:", gnn_f1_normality)
print("SciBERT F1 Normality Test:", scibert_f1_normality)

# Interpretation of Shapiro-Wilk Test results
print("\nInterpretation: GNN F1 scores are" + (" not" if gnn_f1_normality.pvalue < 0.05 else "") + " normally distributed.")
print("Interpretation: SciBERT F1 scores are" + (" not" if scibert_f1_normality.pvalue < 0.05 else "") + " normally distributed.")

# Welch's t-test (does not assume equal variances)
f1_t_test_result = stats.ttest_ind(hetroSciGNN_f1_scores, SciBERT_f1_scores, equal_var=False)

# Print t-test results
print("\nF1 Scores t-test Result:", f1_t_test_result)

# Interpretation of t-test results
print("Interpretation: There is" + (" a" if f1_t_test_result.pvalue < 0.05 else " no") + " statistically significant difference in F1 scores between GNN and SciBERT models.")

In [ ]:
# Sorting scores for CDF
gnn_sorted_scores = np.sort(hetroSciGNN_f1_scores)
scibert_sorted_scores = np.sort(SciBERT_f1_scores)
baseGNN_sorted_scores = np.sort(baseGNN_f1_scores)

# Calculating CDF
gnn_yvals = np.arange(len(gnn_sorted_scores))/float(len(gnn_sorted_scores)-1)
scibert_yvals = np.arange(len(scibert_sorted_scores))/float(len(scibert_sorted_scores)-1)
baseGNN_yvals = np.arange(len(baseGNN_sorted_scores))/float(len(baseGNN_sorted_scores)-1)

# Plotting CDF
plt.figure(figsize=(8, 5))
plt.plot(gnn_sorted_scores, gnn_yvals, label='DruGNNosis-MoA', marker='o')
plt.plot(scibert_sorted_scores, scibert_yvals, label='SciBERT', marker='o')
plt.plot(baseGNN_sorted_scores, baseGNN_yvals, label='Base GNN', marker='o')

plt.xlabel('F1-score')
plt.ylabel('Cumulative Probability')
plt.legend()
plt.grid(True)
plt.savefig('Cumulative_Distribution_F1.png')
plt.show()

In [ ]:
# The folds
folds = np.arange(1, len(hetroSciGNN_f1_scores) + 1)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(folds, baseGNN_sorted_scores, marker='.', lw=3, label='Baseline GNN', linestyle='-',  color='green')
plt.plot(folds, SciBERT_f1_scores, marker='x', lw=3, label='Fine-tuned SciBERT', linestyle='-', color='orange')
plt.plot(folds, hetroSciGNN_f1_scores, marker='o', lw=3, label='DruGNNosis-MoA', linestyle='-', color='blue')

# Adding titles and labels
plt.xlabel('Fold Number', fontsize=16)
plt.ylabel('F1-score', fontsize=16)
plt.xticks(folds)  # Set x-ticks to be fold numbers
plt.legend(fontsize=14)

plt.grid(True)
plt.savefig('Comparison_F1_Scores_3_models.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
# The folds
folds = np.arange(1, len(hetroSciGNN_f1_scores) + 1)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(folds, SciBERT_f1_scores, marker='x', lw=3, label='Fine-tuned SciBERT', linestyle='-', color='orange')
plt.plot(folds, hetroSciGNN_f1_scores, marker='o', lw=3, label='DruGNNosis-MoA', linestyle='-', color='blue')

# Adding titles and labels
plt.xlabel('Fold Number', fontsize=16)
plt.ylabel('F1 Score', fontsize=16)
plt.xticks(folds)  # Set x-ticks to be fold numbers
plt.legend(loc='lower left', fontsize=14)

plt.grid(True)
plt.savefig('Comparison_F1_Scores_2_models.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
hetroSciGNN_cv_df = pd.read_pickle(hetroSciGNN_cv_results_path) # Load data for hetroSciGNN model
baseGNN_cv_df = pd.read_pickle(baseGNN_cv_results_path) # Load data for baseGNN model

# Load data for SciBERT model
with open(SciBERT_valid_fold_metrics_path, 'rb') as file:
    scibert_results = pickle.load(file)

# Extract ROC-AUC scores from the GNN model's test metrics
gnn_roc_auc_scores = hetroSciGNN_cv_df['roc_auc_v']

# Extract ROC-AUC scores from the SciBERT model's test metrics
scibert_roc_auc_scores = np.array([metrics['eval_roc_auc'] for metrics in scibert_results['metrics']])
print(scibert_roc_auc_scores)

# Perform Shapiro-Wilk Test for normality
gnn_roc_auc_normality = stats.shapiro(gnn_roc_auc_scores)
scibert_roc_auc_normality = stats.shapiro(scibert_roc_auc_scores)
print("\nGNN ROC-AUC Normality Test:", gnn_roc_auc_normality)
print("SciBERT ROC-AUC Normality Test:", scibert_roc_auc_normality)

# Interpretation of Shapiro-Wilk Test results
print("\nInterpretation: GNN ROC-AUC scores are" + (" not" if gnn_roc_auc_normality.pvalue < 0.05 else "") + " normally distributed.")
print("Interpretation: SciBERT ROC-AUC scores are" + (" not" if scibert_roc_auc_normality.pvalue < 0.05 else "") + " normally distributed.")

# Perform Welch's t-test (does not assume equal variances)
roc_auc_t_test_result = stats.ttest_ind(gnn_roc_auc_scores, scibert_roc_auc_scores, equal_var=False)
print("\nROC-AUC Scores t-test Result:", roc_auc_t_test_result)

# Interpretation of Welch's t-test results
print("Interpretation: There is" + (" a" if roc_auc_t_test_result.pvalue < 0.05 else " no") + " statistically significant difference in ROC-AUC scores between GNN and SciBERT models.")

## 7.II. Roc-Auc Evaluation

In [ ]:
# Load your data for hetroSciGNN and baseGNN models
hetroSciGNN_cv_df = pd.read_pickle(hetroSciGNN_cv_results_path)
baseGNN_cv_df = pd.read_pickle(baseGNN_cv_results_path)

with open(SciBERT_valid_fold_metrics_path, 'rb') as file:
    scibert_results = pickle.load(file)

num_folds = 10
# Function to compute average ROC curve for GNN models
def compute_avg_roc_gnn(model_df):
    mean_fpr = np.linspace(0, 1, 100)
    mean_tpr = 0.0

    for _, row in model_df.iterrows():
        tpr_interpolated = np.interp(mean_fpr, row['fpr'], row['tpr'])
        mean_tpr += tpr_interpolated

    mean_tpr /= len(model_df)
    mean_tpr[-1] = 1.0  # Ensure it ends at 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    return mean_fpr, mean_tpr, mean_auc

# Compute average ROC curve for hetroSciGNN and baseGNN
mean_fpr_hetroSciGNN, mean_tpr_hetroSciGNN, mean_auc_hetroSciGNN = compute_avg_roc_gnn(hetroSciGNN_cv_df)
mean_fpr_baseGNN, mean_tpr_baseGNN, mean_auc_baseGNN = compute_avg_roc_gnn(baseGNN_cv_df)

# SciBERT - Compute average ROC curve
true_labels = scibert_results['true_labels']
pred_probs = scibert_results['pred_probs']
fold_size = len(true_labels) // (num_folds)
mean_fpr_SciBERT = np.linspace(0, 1, 100)
mean_tpr_SciBERT = 0.0

for i in range(num_folds):
    start_idx = i * fold_size
    end_idx = (i + 1) * fold_size
    fpr, tpr, _ = roc_curve(true_labels[start_idx:end_idx], pred_probs[start_idx:end_idx])
    mean_tpr_SciBERT += np.interp(mean_fpr_SciBERT, fpr, tpr)

mean_tpr_SciBERT /= num_folds
mean_tpr_SciBERT[-1] = 1.0
mean_auc_SciBERT = auc(mean_fpr_SciBERT, mean_tpr_SciBERT)

# Plot the average ROC curves
plt.figure(figsize=(10, 6))
plt.plot(mean_fpr_baseGNN, mean_tpr_baseGNN, color='green', lw=3,label=f'Baseline GNN (ROC-AUC = {mean_auc_baseGNN:.2f})')
plt.plot(mean_fpr_SciBERT, mean_tpr_SciBERT, color='orange', lw=3, label=f'Fine-tuned SciBERT (ROC-AUC = {mean_auc_SciBERT:.2f})')
plt.plot(mean_fpr_hetroSciGNN, mean_tpr_hetroSciGNN, color='blue', lw=3, label=f'DruGNNosis-MoA (ROC-AUC = {mean_auc_hetroSciGNN:.2f})')

plt.plot([0, 1], [0, 1], linestyle='--', lw=3, color='gray', label='Chance')
plt.xlabel('False Positive Rate', fontsize = 16)
plt.ylabel('True Positive Rate', fontsize = 16)
plt.legend(fontsize = 16, loc='lower right')
plt.grid(True)
plt.savefig('3_Models_Roc.pdf')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(mean_fpr_SciBERT, mean_tpr_SciBERT,lw=3, color='orange', label=f'Fine-tuned SciBERT (ROC-AUC = {mean_auc_SciBERT:.4f})')
plt.plot(mean_fpr_hetroSciGNN, mean_tpr_hetroSciGNN,lw=3, color='blue', label=f'DruGNNosis-MoA (ROC-AUC = {mean_auc_hetroSciGNN:.4f})')

# Setting the axis limits to "zoom in" on the graph
plt.xlim(0.0, 0.2)
plt.ylim(0.4, 1.0)

# plt.plot([0, 1], [0, 1], linestyle='--', lw=3, color='gray', label='Chance')

plt.xlabel('False Positive Rate', fontsize = 16)
plt.ylabel('True Positive Rate', fontsize = 16)
plt.legend(loc='lower right', fontsize = 16)
plt.grid(True)
plt.savefig('LLM_GNN_ROC_AUC.pdf')
plt.show()

# 8. Generate Figures

## 8.I. Exploratory Data Analysis

In [ ]:
# Extract rows where Interaction Type is 'drug_target'
data = df_all[df_all['Interaction Type'] == 'drug_target'].drop_duplicates(subset=['Source Name'])

# Define a function for categorizing 'MoA'
def categorize_moa(moa):
    if 'e' in moa:
        return 'Etiological'
    elif 'p' in moa:
        return 'Palliative'
    else:
        return 'Both'

# Apply the function to the 'MoA' column
data['MoA'] = data['MoA'].apply(categorize_moa)
print(data.head())

In [ ]:
categories = [ 'Palliative', 'Etiological', 'Both']

# Define colors for each 'MoA'
colors = [ 'steelblue', 'seagreen', 'goldenrod']

# Count the occurrences of each 'MoA'
moa_counts = data['MoA'].value_counts()
print(moa_counts)
# Calculate percentages for each 'MoA'
moa_percentages = (moa_counts / moa_counts.sum()) * 100
print(moa_percentages)

# Create pie chart with specified colors
fig, ax = plt.subplots()
wedges, texts, autotexts = ax.pie(moa_percentages, colors=colors, autopct='%.3f%%', startangle=35)

# Create the legend for the chart
ax.legend(wedges, categories, title="Categories", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

# Save and show the plot
plt.axis('off')
plt.savefig('EvsP.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
# Define ATC classes as a dictionary
atc_classes = {
   "A": "Alimentary tract and metabolism",
   "B": "Blood and blood forming organs",
   "C": "Cardiovascular system",
   "D": "Dermatologicals",
   "G": "Genito urinary system and sex hormones",
   "H": "Systemic hormonal preparations, excl. sex hormones and insulins",
   "J": "Antiinfectives for systemic use",
   "L": "Antineoplastic and immunomodulating agents",
   "M": "Musculo-skeletal system",
   "N": "Nervous system",
   "P": "Antiparasitic products, insecticides and repellents",
   "R": "Respiratory system",
   "S": "Sensory organs",
   "V": "Various",
   "Others": "Others"
}

# Expand the 'ATC class' column into multiple rows
atc_expanded = data['ATC Class'].str.split(',').apply(pd.Series, 1).stack()
atc_expanded.index = atc_expanded.index.droplevel(-1)
atc_expanded.name = 'ATC_Class'
data_expanded = data.join(atc_expanded)

# Ensure the ATC_Class column only has the letter codes
data_expanded['ATC_Class'] = data_expanded['ATC_Class'].str.strip()

# Group and count occurrences
grouped_data = data_expanded.groupby(['ATC_Class', 'MoA']).size().unstack(fill_value=0)
# Define colors for each 'MoA'
moa_colors = {
    'Both': 'goldenrod',
    'Palliative': 'steelblue',
    'Etiological': 'seagreen',

}

moa_order = ['Etiological', 'Palliative', 'Both']  # Define the order for coloring

# Reorder the columns of grouped_data so that 'Both' is last (for stacking order)
reordered_columns = [col for col in grouped_data if col != 'Both'] + ['Both']
grouped_data = grouped_data[reordered_columns]

# Plot the stacked bar chart
ax = grouped_data.plot(kind='bar', stacked=True,  width=0.8, color=[moa_colors[moa] for moa in moa_order], figsize=(10, 7))
ax.set_xlabel('ATC Class', fontsize=14)
ax.set_ylabel('Frequency', fontsize=14)
# ax.set_title('Frequency of Drugs by ATC Class and MOA') -> # For the paper don't need title for the figure.
ax.set_xticklabels(ax.get_xticklabels(), rotation='horizontal', fontsize=14) # Rotate x-axis labels to vertical

# Create legend for 'MoA'
moa_handles, moa_labels = ax.get_legend_handles_labels()
moa_legend = ax.legend(moa_handles, moa_order, title='MoA', loc='upper left', fontsize=14)
moa_legend.get_title().set_fontsize('large') # Set the font size for the 'MoA' legend title

# Create custom legend for ATC classes without colors
atc_legend_items = [Line2D([0], [0], marker='o', color='w', label=f'{key} - {desc}',
                           markersize=0, markerfacecolor='none') for key, desc in atc_classes.items()]

# Ensure that MoA legend remains visible
ax.add_artist(moa_legend)

# Save and show the plot
plt.savefig('EPvsATC.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show() # Display the chart

## 8.II. Generate Heterogeneous Network Using NetworkX

In [ ]:
# Initialize a NetworkX graph
G = nx.Graph()

# Create a mapping from drug name to MoA label
drug_labels = drug_info_df.set_index('Source Name')['MoA'].to_dict()

# Add nodes
for drug in unique_drugs:
    G.add_node(drug, node_type='drug', MoA=drug_labels.get(drug, 'unknown'))
for disease in unique_diseases:
    G.add_node(disease, node_type='disease')
for gene in all_genes:
    G.add_node(gene, node_type='gene')

# Add edges for Drug-Gene interactions
for src, tgt in zip(source_nodes_dg, target_nodes_dg):
    G.add_edge(unique_drugs[src], all_genes[tgt], edge_type='drug_gene')

# Add edges for Disease-Gene interactions
for src, tgt in zip(source_nodes_pg, target_nodes_pg):
    G.add_edge(unique_diseases[src], all_genes[tgt], edge_type='disease_gene')

# Add edges for Gene-Gene interactions (PPI)
for src, tgt in zip(source_nodes_gg, target_nodes_gg):
    G.add_edge(all_genes[src], all_genes[tgt], edge_type='gene_gene')


In [ ]:
# Now extract the different node sets
drug_nodes_hetero = {node for node, data in G.nodes(data=True) if data.get('node_type') == 'drug'}
disease_nodes_hetero = {node for node, data in G.nodes(data=True) if data.get('node_type') == 'disease'}
gene_nodes_hetero = {node for node, data in G.nodes(data=True) if data.get('node_type') == 'gene'}

In [ ]:
print("Graph Properties:")
print(f"Total number of nodes: {G.number_of_nodes()}")
print(f"Total number of edges: {G.number_of_edges()}")

# Node types: Drug, Disease, Gene
print(f"Number of Drug nodes: {sum(1 for _, attr in G.nodes(data=True) if attr['node_type'] == 'drug')}")
print(f"Number of Disease nodes: {sum(1 for _, attr in G.nodes(data=True) if attr['node_type'] == 'disease')}")
print(f"Number of Gene nodes: {sum(1 for _, attr in G.nodes(data=True) if attr['node_type'] == 'gene')}")

# Edge types: Drug-Gene, Disease-Gene, Gene-Gene
print(f"Number of Drug-Gene edges: {sum(1 for _, _, attr in G.edges(data=True) if attr['edge_type'] == 'drug_gene')}")
print(f"Number of Disease-Gene edges: {sum(1 for _, _, attr in G.edges(data=True) if attr['edge_type'] == 'disease_gene')}")
print(f"Number of Gene-Gene (PPI) edges: {sum(1 for _, _, attr in G.edges(data=True) if attr['edge_type'] == 'gene_gene')}")


## 8.III. Heterogeneous Sub-Graph: Colorectal Cancer

In [ ]:
# Find the highest degree 'disease' node
disease_nodes = [node for node in G.nodes if G.nodes[node]['node_type'] == 'disease']
degree_dict = {node: G.degree(node) for node in disease_nodes}
highest_degree_node = max(degree_dict, key=degree_dict.get)

# Get all neighbors of the highest degree 'disease' node and its neighbors
neighbors = list(G.neighbors(highest_degree_node))
all_neighbors = set(neighbors)
for neighbor in neighbors:
    all_neighbors.update(G.neighbors(neighbor))

# Create a new empty graph to store the subgraph
H = nx.Graph()

# Add 'gene' and 'drug' nodes to the subgraph H that are in all_neighbors
for node_type in ['gene', 'drug']:
    nodes_of_type = [node for node in all_neighbors if G.nodes[node]['node_type'] == node_type][:50]
    for node in nodes_of_type:
        H.add_node(node, **G.nodes[node])

# Add the highest degree 'disease' node to the graph H along with their attributes
H.add_node(highest_degree_node, **G.nodes[highest_degree_node])

# Add the first-level neighbors of the highest degree 'disease' node to the graph H
for neighbor in neighbors:
    H.add_node(neighbor, **G.nodes[neighbor])

# Add the edges between the nodes in H using the edges from the original graph G, excluding self-loops
for edge in G.edges(data=True):
    if edge[0] != edge[1] and edge[0] in H.nodes and edge[1] in H.nodes:
        H.add_edge(edge[0], edge[1], **edge[2])

# Compute the layout for visualization
pos = nx.spring_layout(H, seed=123)

# Set the figure size
plt.figure(figsize=(15, 10))

# Draw nodes with different colors and shapes based on their type
node_colors = ['#1f77b4' if H.nodes[node]['node_type'] == 'gene' else
               '#ff7f0e' if H.nodes[node]['node_type'] == 'drug' else
               '#2ca02c' for node in H.nodes]
node_shapes = ['o' if H.nodes[node]['node_type'] == 'gene' else
               's' if H.nodes[node]['node_type'] == 'drug' else
               '^' for node in H.nodes]

# Draw the nodes
for node_type, shape in [('gene', 'o'), ('drug', 's'), ('disease', '^')]:
    nx.draw_networkx_nodes(H, pos, node_shape=shape,
                           nodelist=[node for node in H.nodes if H.nodes[node]['node_type'] == node_type],
                           node_color=[color for node, color in zip(H.nodes, node_colors) if H.nodes[node]['node_type'] == node_type],
                           node_size=250)

# Draw edges with different colors based on their type
edge_colors = ['#d62728' if 'edge_type' in H.edges[edge] and H.edges[edge]['edge_type'] == 'drug_gene' else
               '#9467bd' if 'edge_type' in H.edges[edge] and H.edges[edge]['edge_type'] == 'disease_gene' else
               '#8c564b' for edge in H.edges]

nx.draw_networkx_edges(H, pos, edge_color=edge_colors, width=0.5)

# Add labels for drugs and diseases
# Draw node labels for drugs with 'e' or 'p' instead of the drug name
drug_labels = {node: ('e' if G.nodes[node]['MoA'] == 1 else 'p') for node in H.nodes() if G.nodes[node]['node_type'] == 'drug'}
nx.draw_networkx_labels(H, pos, labels=drug_labels, font_size=14, font_color='black')

# Draw label for the highest degree phenotype node with smaller font size
label_pos = {highest_degree_node: (pos[highest_degree_node][0], pos[highest_degree_node][1]+0.60)}
nx.draw_networkx_labels(H, label_pos, labels={highest_degree_node: highest_degree_node}, font_size=16, font_color='black')

# Add a smaller arrow annotation pointing to the highest degree phenotype node and positioned a bit higher
plt.annotate('', xy=pos[highest_degree_node], xytext=(pos[highest_degree_node][0], pos[highest_degree_node][1]+0.60),
             arrowprops=dict(facecolor='black', shrink=0.05, width=0.5, headwidth=4))

# Draw edges with specified colors based on their type
edge_colors = []
for u, v, data in H.edges(data=True):
    etype = data.get('edge_type', 'gene_gene')
    if etype == 'drug_gene':
        edge_colors.append('black')
    elif etype == 'disease_gene':
        edge_colors.append('green')
    else:
        edge_colors.append('red')  # Gene-Gene edges are red
nx.draw_networkx_edges(H, pos, edge_color=edge_colors, width=0.5)

# Legends for nodes
gene_patch = Line2D([], [], marker='o', color='w', label='Gene', markersize=14, markerfacecolor='#1f77b4')
drug_patch = Line2D([], [], marker='s', color='w', label='Drug', markersize=14, markerfacecolor='#ff7f0e')
disease_patch = Line2D([], [], marker='^', color='w', label='Disease', markersize=14, markerfacecolor='#2ca02c')
node_legend = plt.legend(handles=[gene_patch, drug_patch, disease_patch], loc='lower left', fontsize='x-large')

# Create a legend for the edge types
edge_legend_items = [
    Line2D([0], [0], color='black', lw=4, label='Drug-Target'),
    Line2D([0], [0], color='green', lw=4, label='Disease-Gene'),
    Line2D([0], [0], color='red', lw=4, label='Gene-Gene')
]

# Add both legends to the plot
plt.gca().add_artist(node_legend)
edge_legend = plt.legend(handles=edge_legend_items, loc='upper left', fontsize='x-large')

# Save and show the plot
plt.axis('off')
plt.savefig('heterogeneous_networks.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()

## 8.IV. Constructing Networks from the Datasets <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Topological Features For PPI & Drug-Target & Disease-Gene Networks.

#### 8.IV.A. Drug-Target Network

In [ ]:
# Extract rows where Interaction Type is 'drug_target'
drug_target_df = df_all[(df_all['Interaction Type'] == 'drug_target')]

unique_drugs = drug_target_df['Source Name'].unique().tolist()
drug_mapping = {drug: i for i, drug in enumerate(unique_drugs)}

unique_genes_drug = drug_target_df['Target Name'].unique().tolist()
genes_drug_mapping = {gene: i for i, gene in enumerate(unique_genes_drug)}

# Drug-Gene interactions based on 'drug_target' interaction type
source_nodes_dg = drug_target_df['Source Name'].map(drug_mapping).dropna().astype(int).values
target_nodes_dg = drug_target_df['Target Name'].map(genes_drug_mapping).dropna().astype(int).values

In [ ]:
# Initialize and build the NetworkX graph
dt_net = nx.Graph()

# Add drug nodes
for drug in unique_drugs:
    dt_net.add_node(drug, node_type='drug')

# Add gene nodes
for gene in unique_genes_drug:
    dt_net.add_node(gene, node_type='gene')

# Add edges for Drug-Gene interactions
for src, tgt in zip(source_nodes_dg, target_nodes_dg):
    dt_net.add_edge(unique_drugs[src], unique_genes_drug[tgt], edge_type='drug_gene')

In [ ]:
# Network statistics
drug_nodes = {node for node, data in dt_net.nodes(data=True) if data.get('node_type') == 'drug'}
gene_nodes = {node for node, data in dt_net.nodes(data=True) if data.get('node_type') == 'gene'}
num_drug_nodes = len(drug_nodes)
num_gene_nodes = len(gene_nodes)
num_edges = dt_net.number_of_edges()

# Connected components
connected_components = list(nx.connected_components(dt_net))
num_connected_components = len(connected_components)
num_isolated_pairs = sum(1 for c in connected_components if len(c) == 2)

# Largest Connected Component (LCC)
largest_cc = max(connected_components, key=len)
LCC_dt = dt_net.subgraph(largest_cc)

# Counting drug and gene nodes in LCC
drug_nodes_LCC = {node for node, data in LCC_dt.nodes(data=True) if data.get('node_type') == 'drug'}
gene_nodes_LCC = {node for node, data in LCC_dt.nodes(data=True) if data.get('node_type') == 'gene'}
num_drug_nodes_LCC = len(drug_nodes_LCC)
num_gene_nodes_LCC = len(gene_nodes_LCC)
num_edges_LCC = LCC_dt.number_of_edges()
num_nodes_LCC = LCC_dt.number_of_nodes()

# Degree analysis in LCC
degree_sequence_LCC = sorted((d for n, d in LCC_dt.degree()), reverse=True)
degree_count_LCC = Counter(degree_sequence_LCC)
most_frequent_degree_LCC = degree_count_LCC.most_common(1)[0][0]

# Output
print(f"Network Statistics:")
print(f"- Number of drug nodes in the entire network: {num_drug_nodes}")
print(f"- Number of gene target nodes in the entire network: {num_gene_nodes}")
print(f"- Number of links (edges) connecting them: {num_edges}")
print(f"- Number of connected components: {num_connected_components}")
print(f"- Number of isolated pairs (standalone drug-target pairs): {num_isolated_pairs}")
print(f"\nLargest Connected Component (LCC):")
print(f"- Total number of nodes in LCC: {num_nodes_LCC}")
print(f"- Number of drug nodes in LCC: {num_drug_nodes_LCC}")
print(f"- Number of gene target nodes in LCC: {num_gene_nodes_LCC}")
print(f"- Number of links (edges) in LCC: {num_edges_LCC}")
print(f"- Most frequent degree in LCC: {most_frequent_degree_LCC}")

##### Generate Figure For The Largest Connected Component of the Drug-Target Network

In [ ]:
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(LCC_dt, seed=24) # Using spring layout for positioning nodes
nx.draw(LCC_dt, pos, with_labels=False, node_size=20, edge_color="gray", linewidths=0.4)
plt.tight_layout()
plt.title("Largest Connected Component of the Drug-Target Network")
plt.show()

#### 8.IV.B. Protein-Protein Interaction (PPI) Network

In [ ]:
# Extract unique genes based on Interaction Type being 'PPI'
PPI_df = df_all[df_all['Interaction Type'] == 'PPI']

# Get unique genes from both 'Source Name' and 'Target Name'
unique_genes_ppi = set(PPI_df['Source Name'].unique()).union(set(PPI_df['Target Name'].unique()))
unique_genes_ppi = list(unique_genes_ppi)  # Convert set back to list
PPI_genes_mapping = {gene: i for i, gene in enumerate(unique_genes_ppi)}

# Gene-Gene interactions (PPI)
source_nodes_gg = PPI_df['Source Name'].map(PPI_genes_mapping).dropna().astype(int).values
target_nodes_gg = PPI_df['Target Name'].map(PPI_genes_mapping).dropna().astype(int).values

Generate PPI Network

In [ ]:
# Initialize a NetworkX graph
PPI = nx.Graph()

# Add gene nodes
for gene in unique_genes_ppi:
    PPI.add_node(gene, node_type='gene')

# Add edges for Gene-Gene interactions (PPI)
for src, tgt in zip(source_nodes_gg, target_nodes_gg):
    PPI.add_edge(unique_genes_ppi[src], unique_genes_ppi[tgt], edge_type='gene_gene')

PPI Topological Features

In [ ]:
# Calculate basic network statistics
num_ppi_nodes = PPI.number_of_nodes()
num_ppi_edges = PPI.number_of_edges()
num_ppi_self_loops = nx.number_of_selfloops(PPI)

# Remove self-loops
PPI.remove_edges_from(nx.selfloop_edges(PPI))

# Identify connected components and calculate statistics
connected_components = list(nx.connected_components(PPI))
num_connected_components = len(connected_components)
num_isolated_nodes = sum(1 for c in connected_components if len(c) == 1)

# Determine the Largest Connected Component (LCC)
largest_cc = max(connected_components, key=len)
LCC_PPI = PPI.subgraph(largest_cc)
num_nodes_LCC = len(largest_cc)
num_edges_LCC = LCC_PPI.number_of_edges()

# Counting gene nodes in the LCC
gene_nodes_LCC = {node for node, data in LCC_PPI.nodes(data=True) if data.get('node_type') == 'gene'}
num_gene_nodes_LCC = len(gene_nodes_LCC)

# Degree analysis in the LCC
degree_sequence = sorted((d for n, d in LCC_PPI.degree()), reverse=True)
degree_count = Counter(degree_sequence)
most_frequent_degree = degree_count.most_common(1)[0][0]

# Calculate the average clustering coefficient in the LCC
avg_clustering_coef = nx.average_clustering(LCC_PPI)

# Output the statistics
print(f"PPI network:")
print(f"- Total number of protein nodes: {num_ppi_nodes}")
print(f"- Total number of edges (excluding self-loops): {num_ppi_edges - num_ppi_self_loops}")
print(f"- Number of self-loop edges: {num_ppi_self_loops}")
print(f"- Number of connected components: {num_connected_components}")
print(f"- Number of isolated protein nodes: {num_isolated_nodes}")
print(f"\nLargest Connected Component (LCC):")
print(f"- Total number of nodes in LCC: {num_nodes_LCC}")
print(f"- Number of gene nodes in LCC: {num_gene_nodes_LCC}")
print(f"- Number of edges in LCC: {num_edges_LCC}")
print(f"- Most frequent degree in LCC: {most_frequent_degree}")
print(f"- Average clustering coefficient in LCC: {avg_clustering_coef}")

##### Generate Figure For The Largest Connected Component of the PPI

In [ ]:
# plt.figure(figsize=(10, 8))
# pos = nx.spring_layout(LCC_PPI, seed=24)  # Using spring layout for positioning nodes
# nx.draw(LCC_PPI, pos, with_labels=False, node_size=20, edge_color="gray", linewidths=0.4)
# plt.tight_layout()
# plt.title("Largest Connected Component of the PPI Network")
# plt.show()

### 8.IV.C. Disease-Gene Network

In [ ]:
# Extract unique genes based on Interaction Type being 'association'
disease_gene_df = df_all[df_all["Interaction Type"] == "association"]

unique_disease = disease_gene_df['Source Name'].unique().tolist()
disease_mapping = {disease: i for i, disease in enumerate(unique_disease)}

unique_gene_association = disease_gene_df['Target Name'].unique().tolist()
gene_association_mapping = {gene: i for i, gene in enumerate(unique_gene_association)}

# Disease-Gene interactions
source_nodes_pg = disease_gene_df['Source Name'].map(disease_mapping).dropna().astype(int).values
target_nodes_pg = disease_gene_df['Target Name'].map(gene_association_mapping).dropna().astype(int).values

Generate Disease-Gene (Diseasome) network

In [ ]:
# Initialize and build the NetworkX graph
diseasome_net = nx.Graph()

# Add disease nodes
for disease in unique_disease:
    diseasome_net.add_node(disease, node_type='disease')

# Add gene nodes
for gene in unique_gene_association:
    diseasome_net.add_node(gene, node_type='gene')

# Add edges for Disease-Gene interactions
for src, tgt in zip(source_nodes_pg, target_nodes_pg):
    diseasome_net.add_edge(unique_disease[src], unique_gene_association[tgt], edge_type='disease_gene')

 Disease-Gene Topological Features

In [ ]:
# Calculate basic network statistics
disease_nodes = {node for node, data in diseasome_net.nodes(data=True) if data.get('node_type') == 'disease'}
gene_nodes = {node for node, data in diseasome_net.nodes(data=True) if data.get('node_type') == 'gene'}
num_disease_nodes = len(disease_nodes)
num_gene_nodes = len(gene_nodes)
num_edges = diseasome_net.number_of_edges()

# Identify connected components and calculate statistics
connected_components = list(nx.connected_components(diseasome_net))
num_connected_components = len(connected_components)
num_isolated_pairs = sum(1 for c in connected_components if len(c) == 2)

# Determine the Largest Connected Component (LCC)
largest_cc = max(connected_components, key=len)
LCC_diseasome = diseasome_net.subgraph(largest_cc)
num_nodes_LCC = len(largest_cc)
num_edges_LCC = LCC_diseasome.number_of_edges()

# Counting disease and gene nodes in the LCC
disease_nodes_LCC = {node for node, data in LCC_diseasome.nodes(data=True) if data.get('node_type') == 'disease'}
gene_nodes_LCC = {node for node, data in LCC_diseasome.nodes(data=True) if data.get('node_type') == 'gene'}
num_disease_nodes_LCC = len(disease_nodes_LCC)
num_gene_nodes_LCC = len(gene_nodes_LCC)

# Degree analysis in the LCC
degree_sequence_LCC = sorted((d for n, d in LCC_diseasome.degree()), reverse=True)
degree_count_LCC = Counter(degree_sequence_LCC)
most_frequent_degree_LCC = degree_count_LCC.most_common(1)[0][0]

# Output the statistics
print(f"Diseasome Network Statistics:")
print(f"- Number of disease nodes in the entire network: {num_disease_nodes}")
print(f"- Number of gene nodes in the entire network: {num_gene_nodes}")
print(f"- Number of links (edges) connecting them: {num_edges}")
print(f"- Number of connected components: {num_connected_components}")
print(f"- Number of isolated pairs (standalone disease-gene pairs): {num_isolated_pairs}")
print(f"\nLargest Connected Component (LCC):")
print(f"- Total number of nodes in LCC: {num_nodes_LCC}")
print(f"- Number of disease nodes in LCC: {num_disease_nodes_LCC}")
print(f"- Number of gene nodes in LCC: {num_gene_nodes_LCC}")
print(f"- Number of links (edges) in LCC: {num_edges_LCC}")
print(f"- Most frequent degree in LCC: {most_frequent_degree_LCC}")

#### Generate Figure For The Largest Connected Component of the Diseasome Network

In [ ]:
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(LCC_diseasome, seed=24)
nx.draw(LCC_diseasome, pos, with_labels=False, node_size=20, edge_color="gray", linewidths=0.4)
plt.tight_layout()
plt.title("Largest Connected Component of the Diseasome Network")
plt.show()

# 9. Realted work - Yildrim

In [ ]:
# Calculate the shortest paths and store in a list
shortest_paths = []

for drug in tqdm(drug_nodes_hetero, desc='Calculate path'):
    drug_moa = G.nodes[drug].get('MoA', 'unknown')  # Retrieve MoA of the drug
    for disease in disease_nodes_hetero:
        try:
            path = nx.shortest_path(G, source=drug, target=disease)
            intermediate_nodes = set(path[1:-1])
            if intermediate_nodes.issubset(gene_nodes_hetero):
                """
                -2 to remove drug and disease from the path length.
                -1 to calculate the distance by the edges.
                for example: path = [DrugA, GeneA, GeneB, GeneC, disease], so the path length is 5.
                But the distance will be 2 because we don't count the drug and disease and calculate the number of edges between GeneA to Gene C.
                """
                path_length = len(path) - 3  # Calculating the path length
                shortest_paths.append((drug, drug_moa, disease, path_length, path))
        except nx.NetworkXNoPath:
            # Record the absence of a path
            shortest_paths.append((drug, drug_moa, None, None, None))

# Filter the shortest paths for each drug
all_shortest_paths = defaultdict(list)
for drug, drug_moa, disease, length, path in shortest_paths:
    if disease is not None:  # If there is a path
        all_shortest_paths[drug].append((drug_moa, disease, length, path))

# Find the shortest path for each drug
unique_shortest_paths = []
for drug, paths in all_shortest_paths.items():
    min_length = min(length for _, _, length, _ in paths)
    for drug_moa, disease, length, path in paths:
        if length == min_length:
            unique_shortest_paths.append((drug, drug_moa, disease, length, path))

# Flatten all_shortest_paths into a list of tuples
arr_all_shortest_paths = []
for drug, paths in all_shortest_paths.items():
    for drug_moa, disease, length, path in paths:
        arr_all_shortest_paths.append((drug, drug_moa, disease, length, path))

Drugs with no path to disease

In [ ]:
drugs_without_paths = drug_nodes_hetero - {drug for drug, _, _, _, _ in shortest_paths if _ is not None}
print(drugs_without_paths)
print(len(drugs_without_paths))

for drug in drugs_without_paths:
    drug_moa = G.nodes[drug].get('MoA', 'unknown')  # Retrieve MoA of the drug
    print(f"{drug}, {drug_moa}")

Save All vs. All dataframe

In [ ]:
# Create DataFrame
df_columns = ['Drug', 'MoA', 'Disease', 'Path Length', 'Path']
df_all = pd.DataFrame(arr_all_shortest_paths, columns=df_columns)

# Save the DataFrame to a compressed csv.gz format
df_all.to_csv('All_shortest_paths_compress.csv.gz', compression='gzip', index=False)
df_all.to_csv('All_shortest_paths.csv', index=False)

Save Unique Shortest Distance dataframe

In [ ]:
# Create DataFrame
df_columns = ['Drug', 'MoA', 'Disease', 'Path Length', 'Path']
df_unique = pd.DataFrame(unique_shortest_paths, columns=df_columns)

# Save the DataFrame to a compressed csv.gz format
df_unique.to_csv('Unique_shortest_paths_compress.csv.gz', compression='gzip', index=False)
df_unique.to_csv('Unique_shortest_paths.csv', index=False)

### 1. **All vs. All Method:** Calculates and counts the shortest distance from each drug to every disease, considering each occurrence of the drug in multiple paths.

In [ ]:
df_all_shortest_paths = pd.read_csv('All_shortest_paths_compress.csv.gz', compression='gzip', low_memory=False)

In [ ]:
print(df_all_shortest_paths.head())

In [ ]:
# Count occurrence for each path length
path_counts = df_all_shortest_paths['Path Length'].value_counts()

# Calculate the fraction for each path length
total_paths = len(df_all_shortest_paths)
fractions = path_counts / total_paths
fractions = fractions.sort_index()

# Initialize the dictionaries
etiological_counts = defaultdict(int)
palliative_counts = defaultdict(int)

# Calculate the counts for each path length and MoA category
moa_counts = df_all_shortest_paths.groupby(['Path Length', 'MoA']).size().unstack(fill_value=0)
print(moa_counts)

# Since 'MoA' contains 0 for etiological and 1 for palliative
etiological_counts = moa_counts[0].to_dict()
palliative_counts = moa_counts[1].to_dict()

# Print to check if the dictionaries are now correctly populated
print(etiological_counts)
print(palliative_counts)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(fractions.index, fractions.values, marker='^', label='Disease genes versus drug targets', color='black', linestyle='-')

# Annotating each point with etiological and palliative
for distance in fractions.index:
    etiological_count = etiological_counts.get(distance, 0)
    palliative_count = palliative_counts.get(distance, 0)

    # Adjust the y_offset to your preference for text placement
    y_offset = 0.0009

    plt.annotate(f'e:{etiological_count}', (distance, fractions[distance] + y_offset), textcoords="offset points", xytext=(0,6), ha='center', fontsize=15)
    plt.annotate(f'p:{palliative_count}', (distance, fractions[distance] - y_offset), textcoords="offset points", xytext=(0,-15), ha='center', fontsize=15)

plt.xlabel('Drug-Disease Distance', fontsize = 16)
plt.ylabel('Fraction of Total Distances', fontsize = 16)
#plt.title('Distribution of Shortest Distance between Drug Target and Disease Gene')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.savefig('Yildrim_all_shortest_distance.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()

### 2. **All vs. All (Unique) Method:** Computes the shortest distance from each drug to every disease, counting each drug only once for each unique path length.

In [ ]:
# Drop duplicates to ensure that each drug is only counted once per distance
df_unique_per_distance = df_all_shortest_paths.drop_duplicates(subset=['Drug', 'Path Length'])

# Count unique occurrences for each path length
unique_path_counts = df_unique_per_distance['Path Length'].value_counts()

# Calculate the fraction for each path length based on unique drug counts
total_unique_drugs = df_unique_per_distance['Drug'].nunique()
unique_fractions = unique_path_counts / total_unique_drugs
unique_fractions = unique_fractions.sort_index()

# Now use unique_fractions for plotting
# Initialize the dictionaries
etiological_counts = defaultdict(int)
palliative_counts = defaultdict(int)

# Calculate the counts for each path length and MoA category
moa_counts = df_unique_per_distance.groupby(['Path Length', 'MoA']).size().unstack(fill_value=0)

# Since 'MoA' contains 0 for etiological and 1 for palliative
etiological_counts = moa_counts[0].to_dict()
palliative_counts = moa_counts[1].to_dict()

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(fractions.index, unique_fractions.values, marker='^', label='Disease genes versus drug targets', color='black', linestyle='-')

# Annotating each point with etiological and palliative
for distance in unique_fractions.index:
    etiological_count = etiological_counts.get(distance, 0)
    palliative_count = palliative_counts.get(distance, 0)

    # Adjust the y_offset to your preference for text placement
    y_offset = 0.0001

    plt.annotate(f'e:{etiological_count}', (distance, unique_fractions[distance] + y_offset), textcoords="offset points", xytext=(0,6), ha='center', fontsize=15)
    plt.annotate(f'p:{palliative_count}', (distance, unique_fractions[distance] - y_offset), textcoords="offset points", xytext=(0,-15), ha='center', fontsize=15)

plt.xlabel('Drug-Disease Distance', fontsize = 16)
plt.ylabel('Fraction of Total Distances', fontsize = 16)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.savefig('Yildrim_ALL_unique_drug_count_shortest_distance.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()

### 3. **Unique Minimum Shortest Distance Method:** For each drug, we determined its shortest distance to a disease. If one drug had the same shortest distance for several diseases, we eliminated the duplicate values, retaining only one unique shortest distance for each drug.

In [ ]:
df_unique_shortest_paths = pd.read_csv('Unique_shortest_paths_compress.csv.gz', compression='gzip')

In [ ]:
df_unique_shortest_paths = df_unique_shortest_paths.drop_duplicates(subset=['Drug', 'Path Length'])

# Count occurrence for each path length
path_counts = df_unique_shortest_paths['Path Length'].value_counts()

# Calculate the fraction for each path length
total_paths = len(df_unique_shortest_paths)
fractions = path_counts / total_paths
fractions = fractions.sort_index()

# Initialize the dictionaries
etiological_counts = defaultdict(int)
palliative_counts = defaultdict(int)

# Calculate the counts for each path length and MoA category
moa_counts = df_unique_shortest_paths.groupby(['Path Length', 'MoA']).size().unstack(fill_value=0)

# Since 'MoA' contains 0 for etiological and 1 for palliative
etiological_counts = moa_counts[0].to_dict()
palliative_counts = moa_counts[1].to_dict()

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(fractions.index, fractions.values, marker='^', label='Disease genes versus drug targets', color='black', linestyle='-')

# Annotating each point with etiological, palliative, and baseline counts
for distance in fractions.index:
    etiological_count = etiological_counts.get(distance, 0)
    palliative_count = palliative_counts.get(distance, 0)

    # Adjust the y_offset to your preference for text placement
    y_offset = 0.0001

    plt.annotate(f'e:{etiological_count}', (distance, fractions[distance] + y_offset), textcoords="offset points", xytext=(0,6), ha='center', fontsize=15)
    plt.annotate(f'p:{palliative_count}', (distance, fractions[distance] - y_offset), textcoords="offset points", xytext=(0,-15), ha='center', fontsize=15)

plt.xlabel('Drug-Disease Distance', fontsize = 16)
plt.ylabel('Fraction of Total Distances', fontsize = 16)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.savefig('Yildrim_unique_minimum_shortest_distance.pdf', format='pdf', dpi=1200, bbox_inches='tight')
plt.show()